In [1]:
import numpy as np

from Input_Variables.read_vars import raw_data_storage, \
                                      mfdfa_wins, mfdfa_q_list, mfdfa_rev_seg, mfdfa_pol_order, \
                                      fpcs_min_time

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.sklearn_pipeline import Sklearn_Pipeline
from Time_Series_Features.time_series_feature_creation import TS_Feature_Creation

### PySpark UDF Schema Activation

In [3]:
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

### Reading In Data Files in PySpark and Pandas

In [4]:
reading_data=Reading_Data(data_location=raw_data_storage)

####### PySpark
pyspark_df=reading_data.read_in_pyspark()
print(pyspark_df.head(1))

####### Pandas
pandas_df=reading_data.read_in_pandas()
print(pandas_df.head(1))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/14 18:53:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/14 18:53:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file

### Sklearn Pipeline in PySpark and Pure Sklearn

In [5]:
pandas_sklearn_pipeline=Sklearn_Pipeline()

####### PySpark
pyspark_transform_schema=pandas_udf_data_schema.sklearn_pyspark_schema()
pyspark_transformations=pandas_sklearn_pipeline.pyspark_sklearn_pipeline(df=pyspark_df, 
                                                                         output_schema=pyspark_transform_schema)
print(pyspark_transformations.head(1))

####### Pandas
pandas_transformations=pandas_sklearn_pipeline.pandas_transform_features(df=pandas_df)
print(pandas_transformations.head(1))

23/02/14 18:53:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


 /Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning:It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.


23/02/14 18:53:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file: file:///Users/carlosmonsivais/Desktop/capstone_data/ahr414_glucose_sample%20-%20ahr414_glucose_sample.csv


[Row(Value=-0.7103890776634216, TrendRate=0.2827904522418976, PatientId='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=', GlucoseDisplayTimeRaw=datetime.datetime(2022, 9, 13, 16, 15, 45), TrendArrow=[0, 1, 0, 0, 0, 0, 0])]
      Value TrendRate                                     PatientId  \
0 -0.710389   0.28279  tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=   

      GlucoseDisplayTimeRaw                           TrendArrow  
0 2022-09-13 19:15:45-04:00  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]  


### Multifractal Detrended Fluctuation Analysis, Poincare Analysis, Functional Principal Component Analysis, Entropy

In [9]:
ts_feature_creation=TS_Feature_Creation()

# # Issue with fcpaWrapper --> Need to make it loop through every PatientID, also fathon library not working on Mac M1 or M2 chips according to documentation
# mfdfa=ts_feature_creation.do_mfdfa(data=pandas_df.Value.to_numpy(), 
#                                    win_sizes=mfdfa_wins, 
#                                    q_list=mfdfa_q_list, 
#                                    rev_seg=mfdfa_rev_seg, 
#                                    pol_order=mfdfa_pol_order)

# Function Works
poincare=ts_feature_creation.poincare_wrapper(data=pandas_transformations)
print(poincare)

# # Issue with fcpaWrapper --> Need to make it loop through every PatientID and how to define minTime variable
# functional_principal_component_analysis=ts_feature_creation.fpcaWrapper(rawData=pandas_transformations, minTime=fpcs_min_time)

# Fuction Works
entropy=ts_feature_creation.entropy_calculation(data=pandas_transformations)
print(entropy)

     SD1    SD2  SD_ratio
1  0.037  0.098     0.373
2  0.268  0.281     0.957
3  0.052  0.083     0.630
    Entropy
0  0.267132
